<a href="https://colab.research.google.com/github/rahimi-raz/nlp/blob/main/CNN_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/alisafaya/OffensEval2020/blob/2e166bcca2cb26f3c8d811fc03f037cca365edf6/bert-cnn.py#L78

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 22.7 MB/s 
     |████████████████████████████████| 182 kB 57.6 MB/s 
     |████████████████████████████████| 7.6 MB 58.0 MB/s 


In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from torch import nn
from models import CNNBert

import unicodedata
import numpy as np
import time
import datetime
import torch
import random
import json
import os
import re
import sys


from transformers import BertConfig, BertTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
 pretrained_model = 'HooshvareLab/bert-fa-base-uncased'

In [ ]:
set_id = sys.argv[1]
use_gpu = False
seed = 1234
max_length = 64
device_ids = [2, 3, 4, 5, 6, 7]
batch_size = 24 * len(device_ids)
lr = 2e-5

In [ ]:
tokenizer = BertTokenizer.from_pretrained(pretrained_model)

Downloading:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440 [00:00<?, ?B/s]

In [ ]:
tokenizer = BertTokenizer.from_pretrained(pretrained_model)
if use_gpu and torch.cuda.is_available():
    device = torch.device("cuda:%d"%(device_ids[0]))
else:
    device = torch.device("cpu")


In [ ]:
def prepare_set(text, max_length=64):
    """returns input_ids, attention_mask, token_type_ids for set of data ready in BERT format"""
    global tokenizer

    t = tokenizer.batch_encode_plus(text,
                        pad_to_max_length=True,
                        add_special_tokens=True,
                        max_length=max_length,
                        return_tensors='pt')

    return t["input_ids"], t["attention_mask"], t["token_type_ids"]

In [ ]:
def predict(model, test_set, batch_size=batch_size):
    test_inputs, test_masks, test_type_ids = prepare_set(test_set)
    test_data = TensorDataset(test_inputs, test_masks, test_type_ids)
    test_sampler = SequentialSampler(test_data)
    test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

    model.eval()
    with torch.no_grad(): 
        preds = []
        for batch in test_dataloader:
            b_input_ids, b_input_mask, b_token_type_ids = tuple(t.to(device) for t in batch)
            y_pred = model(b_input_ids, b_input_mask, b_token_type_ids)
            preds += list(y_pred.cpu().numpy().flatten())

    return preds